In [1]:
import numpy as np
import pandas as pd

In [2]:
# This part of code is the Q learning brain, which is a brain of the agent.
# All decisions are made in here.

In [43]:
# 每个参数的信息可以参考command_line
class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = actions
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)
    
    def check_state_exist(self, state):
        # self.q_table.index self.q_table的name列表
        if state not in self.q_table.index:
            self.q_table = self.q_table.append(
                pd.Series(
                    [0] * len(self.actions),
                    index=self.q_table.columns,
                    name=state
                )
            )
    
    def choose_action(self, observation):
        # q_table 表中不存在就添加到表中
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # 提取要操作的表中的某一行
            state_action = self.q_table.loc[observation, :]
            # 获取数据最大的那列的名字
            action = np.random.choice(
                state_action[state_action == np.max(state_action)].index
            )
        else:
            action = np.random.choice(self.actions)
        return action
    
    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        else:
            q_target = r
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)
        

In [44]:
from maze_env import Maze
def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()
        done = False
        while not done:
            # fresh env
            env.render()
            
            # RL choose action based on observation
            action = RL.choose_action(str(observation))
            
            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)
            
            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_))
            
            # swap observation
            observation = observation_
    # end of game
    print('game over')
    env.destroy()

In [46]:
env = Maze()
RL = QLearningTable(actions=list(range(env.n_actions)))
env.after(100, update)
env.mainloop()